<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/testingsavedmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
#run this
import pandas as pd
import numpy as np
excel_file_path = '/content/preprocessed_data.csv'
column_names = pd.read_csv(excel_file_path, nrows=0).columns.tolist()
df = pd.read_csv(excel_file_path,usecols=[column_names[0],column_names[1]])
df.dropna(inplace=True)
df.head()

,SENTENCES,MEANING
0,how can I commun with my parent,م اپ والدین سے کیسا با کرنا
1,how can I make friend,م دوس کیسا بنائ
2,whi do I get so sad,م اتنا اداس کم ہ
3,if youv ask yourself such question your not alon,اگر اپ اپ اپ سے ایسا سوالا کرنا ہ ت اپ اکیلے نہ ہ
4,depend on where youv turn for guidanc you may ...,میں با پرنا منحصر ہونا کہنا اپ رہنمائ کم لینا ...


In [ ]:
#dont run
train,val,test=np.split(df.sample(frac=1),[int(0.8*len(df)),int(0.9*len(df))])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
vocab_size=7000
embedding_dim=50
max_length=500
trunc_type='post'
oov_tok='<OOV>'
test_inputs=test['SENTENCES '].values
test_inputs_str=[str(x) for x in test_inputs]
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(test_inputs_str) # Fit the tokenizer on the text data first
test_sequences = tokenizer.texts_to_sequences(test_inputs_str)

# Filter out None values from the sequences
test_sequences = [[x for x in seq if x is not None] for seq in test_sequences]


test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

test_labels=test['MEANING'].values
test_labels = [str(x) for x in test_labels]
tokenizer.fit_on_texts(test_labels)
test_labels_sequences = tokenizer.texts_to_sequences(test_labels)
padded_test_labels = pad_sequences(test_labels_sequences, maxlen=max_length, truncating=trunc_type)

In [ ]:
test

,SENTENCES,MEANING
12673,but he go out and begin to publish it much and...,لیکن میں باہر جاکر بہ چرچا کرنا لگنا اور میں ب...
7270,take I just one visit to make their tenderloin...,میں کم ٹینڈرلوئن اسٹیک طوفان کو کراچ م اسٹیک پ...
29204,good food,اچھا کھا
3990,I have not hear much about it,م میں کم بارے م زیادہ نہ سننا ہونا ۔
12395,behold I have tell you befor,دیکھ م پہلا ہ میں سے کہہ دیا ہونا ۔
...,...,...
6884,it wa an okay experi a bit surpris to see an i...,میں ایک ٹھیک تجربہ تھا مینہ م جد دیکھ کرنا قدر...
11755,and everi one that heareth these say of mine a...,اور جو کوئ میر میں بات سن اور میں پرنا عمل نہ ...
8759,nihari have a hair strand in it and serious it...,نہار کم میں م بال ک پٹنا تھ اور سنجیدگ سے میں ...
6351,becaus their special peri peri bite not avail ...,"کیونکہ میں ک خصوصی ""پیر پیر بائٹ"" اج دستیاب نہ..."


Testing lstm model

In [21]:
model=tf.keras.models.load_model('/content/Lstmmodel15k800ep2.keras')

In [22]:
import pickle
with open('/content/tokenizer15k800.pickle', 'rb') as handle:
    tokenizer158k = pickle.load(handle)

In [23]:
import numpy as np

In [ ]:
model.evaluate(test_padded,padded_test_labels)

93/93 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.9704 - loss: 0.2709


[0.26863566040992737, 0.9704049825668335]

In [25]:
new_data = ["how can I commun with my parent"]
vocab_size=15000
embedding_dim=50
max_length=700
trunc_type='post'
oov_tok='<OOV>'
tokenizer158k.fit_on_texts(new_data) # Update the tokenizer's vocabulary
new_sequences = tokenizer158k.texts_to_sequences(new_data) # Get the sequences
new_padded = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_length, truncating=trunc_type)
# Make predictions
predictions = model.predict(new_padded)
# View the prediction result
print(predictions)
predicted_sequence = np.argmax(predictions, axis=-1)
# Convert the predicted sequence of indices back to words
predicted_sentence = [tokenizer.index_word.get(index, '') for index in predicted_sequence[0]]
# Join the words to form a sentence
predicted_sentence_str = ' '.join(predicted_sentence)
print(predicted_sentence_str)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
[[[8.4266043e-01 5.4957764e-04 8.2908361e-04 ... 1.7403804e-06
   1.6660920e-06 1.6641641e-06]
  [9.9297911e-01 4.2341289e-05 6.6708191e-05 ... 2.4078597e-08
   2.2270420e-08 2.2529326e-08]
  [9.9564022e-01 2.7835584e-05 4.4284574e-05 ... 1.1976427e-08
   1.1019405e-08 1.1167282e-08]
  ...
  [3.5532635e-02 8.5388152e-03 4.9569536e-02 ... 2.9501753e-06
   2.9503949e-06 2.9993930e-06]
  [3.4835111e-02 8.5505182e-03 4.9802795e-02 ... 2.9494151e-06
   2.9500254e-06 2.9990204e-06]
  [3.3832774e-02 8.5088676e-03 4.9318634e-02 ... 3.0349518e-06
   3.0370013e-06 3.0866752e-06]]]
<oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov>

In [27]:
import numpy as np
import tensorflow as tf
import pickle

# Load the pretrained model
model = tf.keras.models.load_model('/content/Lstmmodel15k800ep2.keras')

# Load the tokenizer
with open('/content/tokenizer15k800.pickle', 'rb') as handle:
    tokenizer158k = pickle.load(handle)

# Prepare multiple new data inputs

# Preprocessed input data
new_data = [
    "how can i communicate better with my parent",
    "what be some good way to resolve conflict",
    "how to improve my relationship with my father",
    "what should i do if i feel misunderstood",
    "how can i express my feeling effectively",
    "what be some tip for talking to my mom",
    "how to ask for advice from my parent",
    "way to build trust with my family",
    "how can i discuss sensitive topic with my dad",
    "tip for have open conversation with my child"
]
# Preprocess the input data

# Define parameters
vocab_size = 15000
embedding_dim = 50
max_length = 700
trunc_type = 'post'
oov_tok = '<OOV>'

# Process and predict for each new data input
for text in new_data:
    new_sequences = tokenizer158k.texts_to_sequences([text])  # Get the sequence for each text
    new_padded = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_length, truncating=trunc_type)

    # Make predictions
    predictions = model.predict(new_padded)

    # View the prediction result
    print(f"Input (preprocessed): {text}")
    print(f"Predictions: {predictions}")

    # Get the predicted sequence of indices
    predicted_sequence = np.argmax(predictions, axis=-1)

    # Convert the predicted sequence of indices back to words
    predicted_sentence = [tokenizer158k.index_word.get(index, '') for index in predicted_sequence[0]]

    # Join the words to form a sentence
    predicted_sentence_str = ' '.join(predicted_sentence)
    print(f"Predicted Sentence: {predicted_sentence_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
Input (preprocessed): how can i communicate better with my parent
Predictions: [[[8.4266043e-01 5.4957764e-04 8.2908361e-04 ... 1.7403804e-06
   1.6660920e-06 1.6641641e-06]
  [9.9297911e-01 4.2341289e-05 6.6708191e-05 ... 2.4078597e-08
   2.2270420e-08 2.2529326e-08]
  [9.9564022e-01 2.7835584e-05 4.4284574e-05 ... 1.1976427e-08
   1.1019405e-08 1.1167282e-08]
  ...
  [3.4716561e-02 8.5344622e-03 4.9589638e-02 ... 2.9757823e-06
   2.9766709e-06 3.0258311e-06]
  [3.4359340e-02 8.5421726e-03 4.9734164e-02 ... 2.9729247e-06
   2.9739967e-06 3.0231531e-06]
  [3.3573382e-02 8.5038906e-03 4.9278893e-02 ... 3.0485094e-06
   3.0507922e-06 3.1005025e-06]]]
Predicted Sentence:                                                                                                                                                                                                                                                                                             